This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv('/Volumes/My Passport for Mac/Springboard stuff/dsc/Unit 7 - Data Wrangling/7.2 - Data Collection/7.2.5 - API Mini Project/.env_example')
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
res = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=dict(api_key=API_KEY))

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [4]:
import json

In [5]:
# converts the json object into a python dictionary
json_data = res.json()

In [6]:
# finding the column names (AKA the keys) for the data
print(json_data['dataset_data'].keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [7]:
json_data['dataset_data']['data'][:5]

[['2020-12-01',
  112.2,
  112.2,
  111.5,
  112.0,
  None,
  51.0,
  5703.0,
  None,
  None,
  None],
 ['2020-11-30',
  111.0,
  113.6,
  111.0,
  112.1,
  None,
  315.0,
  35111.5,
  None,
  None,
  None],
 ['2020-11-27',
  111.0,
  111.5,
  109.5,
  111.2,
  None,
  155.0,
  17168.5,
  None,
  None,
  None],
 ['2020-11-26',
  111.3,
  112.0,
  110.5,
  111.6,
  None,
  254.0,
  28367.5,
  None,
  None,
  None],
 ['2020-11-25',
  111.4,
  111.4,
  110.1,
  110.1,
  None,
  294.0,
  32435.8,
  None,
  None,
  None]]

In [8]:
json_data['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [9]:
# isolate the 2017 data
actual_data = json_data['dataset_data']['data']
year_2017_data = []
for items in actual_data:
    if items[0][:4] == '2017':
        year_2017_data.append(items)
print(year_2017_data[0])

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [10]:
# find the highest high and the lowest high
highs_list = []
for items in year_2017_data:
    highs_list.append(items[2])
print("The highest high for the year 2017 was {}, and the lowest high for the year 2017 was {}.".format(max(highs_list), min(highs_list)))


The highest high for the year 2017 was 53.54, and the lowest high for the year 2017 was 34.12.


In [11]:
# largest change in any one day based on High and Low price:
daily_change_list = []
for items in year_2017_data:
    daily_change = abs(items[2]-items[3])
    daily_change_list.append(daily_change)
print("The largest change in any one day based on High and Low price was {}.".format(max(daily_change_list)))

The largest change in any one day based on High and Low price was 2.8100000000000023.


In [12]:
# largest change in any two days based on closing price:
ordered_2017_data = year_2017_data[::-1]
ordered_2017_data_shift = ordered_2017_data[1:]
two_day_change_list = []

for i in range(len(ordered_2017_data_shift)):
    two_day_change = abs(ordered_2017_data[i][4]-ordered_2017_data_shift[i][4])
    two_day_change_list.append(two_day_change)
print("The largest change in any two days based on closing prices was {}.".format(max(two_day_change_list)))

The largest change in any two days based on closing prices was 2.559999999999995.


In [13]:
# average daily trading volume for 2017:
tvol_list = []

for items in year_2017_data:
    tvol_list.append(items[6])
print("The average daily trading volume for 2017 was {}.".format(sum(tvol_list)/float(len(tvol_list))))

The average daily trading volume for 2017 was 89124.33725490196.


In [14]:
# definition of median function from https://stackoverflow.com/questions/24101524/finding-median-of-list-in-python
def median(data):
    data.sort()
    mid = len(data) // 2
    return (data[mid] + data[~mid]) / 2.0

In [15]:
# median trading volume:
print("The median trading volume for 2017 was {}.".format(median(tvol_list)))

The median trading volume for 2017 was 76286.0.
